In [2]:
import pandas as pd

In [3]:
import os
for dirname, _, filenames in os.walk('./archive'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [4]:
dfs = [pd.read_csv(f, index_col=[0], parse_dates=[0])
        for f in os.listdir(os.getcwd()) if f.endswith('csv')]

t = pd.concat(dfs, axis=0, join='inner').sort_index()

**Q1:** Estimate the average number of people that 'like' any given tweet by Elon Musk. 

In [5]:
round(t.nlikes.sum()/t.shape[0])

14705

**Q2:** from the above, estimate how many people view one of his tweets, on average. Assume that engagement rate, calculated by summing likes, replies, and retweets, and dividing by views, is roughly .05 for any given tweet.


In [6]:
# Engagement in total
v_total = sum(t.nlikes+t.nreplies+t.nretweets)/0.05
# Estimation of engagement per tweet
v_per = v_total/t.shape[0]
round(v_per)

338864

**Q3**: Estimate the average amount of data (in bytes) stored for each Elon Musk tweet. 

Here consider data consist of two aspects: 1. tweet(text+attached media) 2. info of that tweet (e.g id, nlikes, link...)


In [7]:
# Compute the first part: tweet content(text + attached media file)
# check the format of the thumbnail field to specify the amount of video and images, respectively.
sr=list(t.thumbnail[t.thumbnail.notna()==True])
r_list = []
for i in range(len(sr)):
     key=sr[i].split('/')[3]
     r_list.append(key)
i=0
video_num=len([i+1 for r in r_list if "video_thumb" in r])
img_num=len(sr)-video_num


Assume average media files in twitter is 45 second and 15mb(video), 16-bit color with 1024*512 ptx(images) and 28 UTF-8 characters content

In [8]:
media_avg_size= (video_num*15*1048576+img_num*1024*512*16/8+len(t.tweet)*28)/len(t.id)

# Compute the second part: info
# Randomly take a sample to get the amount of info data
import random, sys
random.seed(2022)
info_avg_size=sys.getsizeof(t.iloc[random.randint(0,len(t.id))])

# Get the combined average size of an Elon tweet
avg_data_stored = info_avg_size + media_avg_size
round(avg_data_stored)

159646

**Q4:** Using the answers above, estimate the total data transfer involved in displaying any given Elon Musk tweet to Twitter users.

In [12]:
# Assume Twitter will display all the data stored to users
display_data = avg_data_stored*len(t.id)
round(display_data)

6876602216

**Q5:** Estimate how many Elon tweets are viewed, per-minute, by the site's userbase.

Assumption: userbase is the engaement of Musk' tweets calculated in Q2 with a time span around 11 years

In [10]:
total_minutes = 11*365*24*60
v_per_minute = v_total/total_minutes
round(v_per_minute)

2525

**Q6:** Using the above, estimate how much data is accessed per-minute by Twitter in the process of displaying Elon Musk tweets.

Assumption: that is, view per minute times average amount of data when displaying an Elon tweet

In [11]:
round(v_per_minute*display_data)

403043227

**Q7:** choose a function that involves modifying a tweet (say, updating its 'likes' 
count, which you can assume is cached in a tweet's associated database row). 
How much data do you believe is touched by this operation on a monthly basis? 
Consider the amount of data affected, the frequency at which the operation is carried out, and so on.


**Function:** Updating 'likes' count <br>
The data column affected: nlikes <br>
The frequency of this operation: monthly views(engagement) * chance to click the 'likes' bottom

In [294]:
likes_basis = sys.getsizeof(t.nlikes) # size of nlikes in bytes
v_monthly = (v_total*0.05)/(11*24)  # suppose the whole time span is 11 years and 0.05 rate of participation
data_touch_mons = likes_basis*v_monthly
data_touch_mons